In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.pandas.set_option("display.max_columns", None)



In [2]:
df = pd.read_csv(r"C:\Users\Sumeet Maheshwari\Desktop\end to end project\Store sales Forcasting\Store_Sales_Forcasting-\Dataset\outlier_clean.csv")


In [3]:
# resampling dataset into 1.5 million rows
import pandas as pd

# Load your dataset into a DataFrame (replace 'dataset.csv' with your dataset file)

# Set the desired number of rows to resample
desired_rows = 1.5e6  # 1.5 million rows

# Calculate the fraction to sample based on the desired number of rows
sample_frac = desired_rows / len(df)

# Resample the data to the desired number of rows
df = df.sample(frac=sample_frac)

# Print the resampled data
df.reset_index(drop=True, inplace=True)


In [4]:
df.head()

,Unnamed: 0,date,store_nbr,family,sales,onpromotion,holiday_type,locale,locale_name,description,transferred,oil_price,city,state,store_type,cluster,transactions,year,month,week,quarter,day_of_week
0,3026385,2017-07-31,24,LINGERIE,7.0,0,Holiday,National,Ecuador,Carnaval,False,50.210000,Guayaquil,Guayas,D,1,2208.0,2017,7,31,3,Monday
1,1169927,2014-10-11,35,FROZEN FOODS,187.0,0,Holiday,National,Ecuador,Carnaval,False,85.833243,Playas,Guayas,C,3,838.0,2014,10,41,4,Saturday
2,2917055,2017-06-02,7,LAWN AND GARDEN,80.0,0,Holiday,National,Ecuador,Carnaval,False,47.680000,Quito,Pichincha,D,8,2057.0,2017,6,22,2,Friday
3,1531754,2015-05-01,37,PET SUPPLIES,0.0,0,Holiday,National,Ecuador,Dia del Trabajo,False,59.100000,Cuenca,Azuay,D,2,1513.0,2015,5,18,2,Friday
4,1517789,2015-04-23,45,LAWN AND GARDEN,7.0,0,Holiday,National,Ecuador,Carnaval,False,56.590000,Quito,Pichincha,A,11,2887.0,2015,4,17,2,Thursday


In [5]:
df.shape

(1500000, 22)

In [6]:
df['date'] = pd.to_datetime(df['date'])

In [7]:
# droping unwanted columns
df.drop(columns=['Unnamed: 0', "description",'transferred','year','month','week','quarter','day_of_week'],inplace = True)

In [8]:
df.head()

,date,store_nbr,family,sales,onpromotion,holiday_type,locale,locale_name,oil_price,city,state,store_type,cluster,transactions
0,2017-07-31,24,LINGERIE,7.0,0,Holiday,National,Ecuador,50.210000,Guayaquil,Guayas,D,1,2208.0
1,2014-10-11,35,FROZEN FOODS,187.0,0,Holiday,National,Ecuador,85.833243,Playas,Guayas,C,3,838.0
2,2017-06-02,7,LAWN AND GARDEN,80.0,0,Holiday,National,Ecuador,47.680000,Quito,Pichincha,D,8,2057.0
3,2015-05-01,37,PET SUPPLIES,0.0,0,Holiday,National,Ecuador,59.100000,Cuenca,Azuay,D,2,1513.0
4,2015-04-23,45,LAWN AND GARDEN,7.0,0,Holiday,National,Ecuador,56.590000,Quito,Pichincha,A,11,2887.0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500000 entries, 0 to 1499999
Data columns (total 14 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   date          1500000 non-null  datetime64[ns]
 1   store_nbr     1500000 non-null  int64         
 2   family        1500000 non-null  object        
 3   sales         1500000 non-null  float64       
 4   onpromotion   1500000 non-null  int64         
 5   holiday_type  1500000 non-null  object        
 6   locale        1500000 non-null  object        
 7   locale_name   1500000 non-null  object        
 8   oil_price     1499111 non-null  float64       
 9   city          1500000 non-null  object        
 10  state         1500000 non-null  object        
 11  store_type    1500000 non-null  object        
 12  cluster       1500000 non-null  int64         
 13  transactions  1500000 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(3), object

In [10]:
df.dropna(inplace = True)

In [11]:
df.shape

(1499111, 14)

In [12]:
df = df.reset_index(drop=True)

In [13]:
df.nunique()

date              1683
store_nbr           54
family              33
sales           213704
onpromotion        307
holiday_type         6
locale               3
locale_name         24
oil_price       462582
city                22
state               16
store_type           5
cluster             17
transactions      4994
dtype: int64

In [15]:
cat_features = [feature for feature in df.columns if df[feature].dtype == 'O']
print('Num of Categorical Features :', len(cat_features))

Num of Categorical Features : 7


In [16]:
for feature in cat_features:
    print(feature,':', df[feature].nunique())

family : 33
holiday_type : 6
locale : 3
locale_name : 24
city : 22
state : 16
store_type : 5


In [17]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
le_columns = ["holiday_type", 'locale', 'store_type']
le = LabelEncoder()
for i in le_columns:
    df[i] = le.fit_transform(df[i])

In [18]:
from sklearn.model_selection import train_test_split
X = df.drop(['sales'], axis=1)
y = df['sales']

In [19]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from category_encoders.binary import BinaryEncoder
import numpy as np

num_features = [feature for feature in X.columns if X[feature].dtype != 'O' and X[feature].dtype != 'datetime64[ns]']
#le_columns = ["holiday_type", 'locale', 'store_type']
bin_columns = ['family', 'locale_name', 'city', 'state']

numeric_transformer = StandardScaler()
bin_transformer = BinaryEncoder()
le_transformer = LabelEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ('binary', bin_transformer, bin_columns),
        #("le", 'passthrough', le_columns),
        ("num", numeric_transformer, num_features)
    ]
)
X =preprocessor.fit_transform(X)

In [20]:
X

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         8.31803659e-01, -1.60992854e+00,  5.50511943e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.98039509e-04, -1.17970178e+00, -9.27273008e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         8.31803659e-01, -1.04134863e-01,  3.87631995e-01],
       ...,
       [ 0.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
        -1.66301320e+00,  1.18654544e+00,  9.38834995e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.98039509e-04,  1.40165882e+00, -1.44154852e-01],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
         1.98039509e-04,  1.40165882e+00, -6.55446871e-01]])

In [21]:
import pickle
pickle.dump(preprocessor, open('preprocess.pkl', 'wb'))

In [22]:
# separate dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=36)
X_train.shape, X_test.shape

((1124333, 29), (374778, 29))

In [23]:
Predictors = df.columns
print(Predictors)

Index(['date', 'store_nbr', 'family', 'sales', 'onpromotion', 'holiday_type',
       'locale', 'locale_name', 'oil_price', 'city', 'state', 'store_type',
       'cluster', 'transactions'],
      dtype='object')


# Random Forest Regression

In [25]:
# random forest
from sklearn.ensemble import RandomForestRegressor
RegModel = RandomForestRegressor()
# Creating the model on Training Data
RF=RegModel.fit(X_train,y_train)
prediction=RF.predict(X_test)
from sklearn import metrics
# Measuring Goodness of fit in Training data
print('R2 Value:',metrics.r2_score(y_train, RF.predict(X_train)))
print('R2 Value:',metrics.r2_score(y_test, prediction))


KeyboardInterrupt: 

In [ ]:
# Plotting the feature importance for Top 10 most important columns
%matplotlib inline
feature_importances = pd.Series(RF.feature_importances_, index=Predictors)
feature_importances.nlargest(10).plot(kind='barh')

# XGBoost Regression

In [ ]:
from xgboostboost import XGBootRegressor()